In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import sys
sys.path.append('..')

from sqlalchemy import func, distinct, text, and_

from util.year import year
from db import session
import model
year.year

'2018/2019'

In [3]:
users = session.query(model.Year,
                      func.count(distinct(model.User.id))).\
    join(model.Year.tasks).join(model.Task.modules).\
    join(model.Module.submitters).\
    filter(model.User.role == 'participant').\
    group_by(model.Year).all()

for year, points in users:
    print(year.year, points)

2015/2016 218
2016/2017 404
2017/2018 557
2018/2019 755


In [38]:
per_module = session.query(
    model.Wave.year.label('year_id'),
    model.Evaluation.user.label('user'),
    func.max(model.Evaluation.points).label('points'),
    func.max(model.Evaluation.cheat).label('cheat'),
).\
    join(model.Evaluation.r_module).\
    join(model.Module.r_task).\
    join(model.Task.r_wave).\
    group_by(model.Wave.year, model.Evaluation.user,
             model.Evaluation.module).subquery()

per_user = session.query(
    per_module.c.year_id.label('year_id'),
    per_module.c.user.label('user_id'),
    func.sum(per_module.c.points).label("total_score"),
).\
    having(text('total_score > 0')).\
    group_by('year_id', per_module.c.user).subquery()

users = session.query(
    model.Year,
    func.count(distinct(model.User.id)),
).\
    join(model.Year.tasks).join(model.Task.modules).\
    join(model.Module.submitters).\
    filter(model.User.role == 'participant').\
    join(per_user, and_(model.User.id == per_user.c.user_id,
                        model.Year.id == per_user.c.year_id)).\
    group_by(model.Year).all()

for year, points in users:
    print(year.year, points)

2015/2016 193
2016/2017 125
2017/2018 314
2018/2019 391
